In [68]:
import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import os
POSTGRES_PASSWORD= os.getenv('POSTGRES_PASSWORD')

In [142]:
bills=pd.read_csv('Data/bills.csv')  
committee_members=pd.read_csv('Data/committee_members.csv') 
committees=pd.read_csv('Data/committees.csv')  
congress_members=pd.read_csv('Data/congress_members.csv') 
leadership=pd.read_csv('Data/leadership.csv')  
os_contributions=pd.read_csv('Data/os_contributions.csv') 
os_legislators=pd.read_csv('Data/os_legislators.csv') 
terms=pd.read_csv('Data/terms.csv')  
votingaffinity=pd.read_csv('Data/votingaffinity.csv')

# Bills

In [143]:
bills.columns= [x.lower() for x in bills.columns]
bills.columns= [x.replace('.','_') for x in bills.columns]
bills.columns

Index(['congress', 'introduceddate', 'number', 'title', 'type', 'url',
       'latestaction_actiondate', 'latestaction_text', 'policyarea_name',
       'amendmentnumber', 'latestaction', 'latestaction_actiontime'],
      dtype='object')

In [144]:
bills.head(3).T

,0,1,2
congress,118,118,118
introduceddate,2023-10-04,2023-09-29,2023-09-27
number,405.0,2998.0,373.0
title,A resolution expressing support for the design...,Land-Grant Research Equity and Accountability Act,A resolution designating the week of September...
type,SRES,S,SRES
url,https://api.congress.gov/v3/bill/118/sres/405?...,https://api.congress.gov/v3/bill/118/s/2998?fo...,https://api.congress.gov/v3/bill/118/sres/373?...
latestaction_actiondate,2023-10-04,2023-09-29,2023-09-27
latestaction_text,"Submitted in the Senate, considered, and agree...",Read twice and referred to the Committee on Ag...,"Submitted in the Senate, considered, and agree..."
policyarea_name,Armed Forces and National Security,Agriculture and Food,NaN
amendmentnumber,NaN,NaN,NaN


In [145]:
congress_members.columns= [x.lower() for x in congress_members.columns]
congress_members.columns= [x.replace('.','_') for x in congress_members.columns]

congress_members.columns



Index(['bioguideid', 'birthyear', 'currentmember', 'directordername',
       'firstname', 'honorificname', 'invertedordername', 'lastname',
       'officialwebsiteurl', 'partyhistory', 'state', 'terms', 'updatedate',
       'addressinformation_city', 'addressinformation_district',
       'addressinformation_officeaddress', 'addressinformation_phonenumber',
       'addressinformation_zipcode', 'cosponsoredlegislation_count',
       'cosponsoredlegislation_url', 'depiction_attribution',
       'depiction_imageurl', 'sponsoredlegislation_count',
       'sponsoredlegislation_url', 'middlename', 'suffixname', 'nickname',
       'leadership', 'district'],
      dtype='object')

# congress_members

In [146]:
congress_members=congress_members.drop(['terms', 'leadership', 'partyhistory'], axis=1)

In [147]:
congress_members.head(3).T

,0,1,2
bioguideid,B000944,C000127,C000141
birthyear,1952,1958,1943
currentmember,True,True,True
directordername,Sherrod Brown,Maria Cantwell,Benjamin L. Cardin
firstname,Sherrod,Maria,Ben
honorificname,Mr.,Ms.,Mr.
invertedordername,"Brown, Sherrod","Cantwell, Maria","Cardin, Benjamin L."
lastname,Brown,Cantwell,Cardin
officialwebsiteurl,https://www.brown.senate.gov/,https://www.cantwell.senate.gov,https://www.cardin.senate.gov/
state,Ohio,Washington,Maryland


# Terms

In [148]:
terms.columns= [x.lower() for x in terms.columns]
terms.columns= [x.replace('.','_') for x in terms.columns]

terms.columns

Index(['chamber', 'congress', 'endyear', 'membertype', 'startyear',
       'statecode', 'statename', 'bioguideid', 'district'],
      dtype='object')

In [149]:
terms.head(3).T

,0,1,2
chamber,Senate,Senate,Senate
congress,107,108,109
endyear,2003.0,2005.0,2007.0
membertype,Senator,Senator,Senator
startyear,2002,2003,2005
statecode,TX,TX,TX
statename,Texas,Texas,Texas
bioguideid,C001056,C001056,C001056
district,NaN,NaN,NaN


In [150]:
leadership.columns= [x.lower() for x in leadership.columns]
leadership.columns= [x.replace('.','_') for x in leadership.columns]

leadership.columns

Index(['congress', 'type', 'bioguideid', 'current'], dtype='object')

In [151]:
leadership.head(3).T

,0,1,2
congress,113,114,115
type,Assistant Democratic Leader,Assistant Majority Leader,Majority Whip
bioguideid,C001056,C001056,C001056
current,NaN,NaN,NaN


# os_legislators

In [152]:
os_legislators.columns= [x.lower() for x in os_legislators.columns]
os_legislators.columns= [x.replace('.','_') for x in os_legislators.columns]

os_legislators.columns

Index(['cid', 'firstl', 'lastnam', 'party', 'offic', 'gend', 'first_elected',
       'xit_cod', 'commen', 'phon', 'fax', 'w', 'webform', 'congress_offic',
       'oguide_id', 'votesmart_id', 'feccandid', 'witter_id', 'youtube_url',
       'facebook_id', 'hd'],
      dtype='object')

In [153]:
os_legislators.head(3).T

,0,1,2
cid,N00050780,N00035774,N00026050
firstl,Mary Peltola,Dan Sullivan,Lisa Murkowski
lastnam,Peltola,Sullivan,Murkowski
party,D,R,R
offic,AK01,AKS1,AKS2
gend,F,M,F
first_elected,2022,2014,2002
xit_cod,0,0,0
commen,NaN,NaN,NaN
phon,NaN,202-224-3004,202-224-6665


In [154]:
os_legislators = os_legislators[['cid', 'party', 'oguide_id']]
os_legislators = os_legislators.rename({'oguide_id': 'bioguideid'}, axis =1)

os_legislators

,cid,party,bioguideid
0,N00050780,D,NaN
1,N00035774,R,S001198
2,N00026050,R,M001153
3,N00044245,R,C001054
4,N00041295,R,M001212
...,...,...,...
532,N00032838,D,M001183
533,N00009771,R,C001047
534,N00049197,R,H001096
535,N00006236,R,B001261


members = pd.merge(congress_members, os_legislators, on = 'bioguideid',
                  how = 'outer',
                  validate = 'one_to_many',
                  indicator = 'matched')

In [155]:
#members['matched'].value_counts()

In [156]:
# bio= congress_members.query("lastname =='peltola'").reset_index()
# ['bioguideid'][0]

In [157]:
members = pd.merge(congress_members, os_legislators, on = 'bioguideid',
                  how = 'inner',
                  )

In [158]:
os_contributions.columns= [x.lower() for x in os_contributions.columns]
os_contributions.columns= [x.replace('.','_') for x in os_contributions.columns]

os_contributions.columns

Index(['org_nam', 'otal', 'pac', 'ndiv'], dtype='object')

# os_contributions

In [159]:
os_contributions

,org_nam,otal,pac,ndiv
0,State of Alaska,19541,0,19541
1,"Sonosky, Chambers et al",16050,0,16050
2,General Communication Inc,15150,2500,12650
3,Google Inc,14164,0,14164
4,University of Alaska/Anchorage,13938,0,13938
...,...,...,...,...
1995,New York Cancer & Blood Specialists,15000,0,15000
1996,Transdigm Group,15000,10000,5000
1997,Altria Group,12900,10000,2900
1998,LHC Group,11800,5000,6800


In [160]:
committees.columns= [x.lower() for x in committees.columns]
committees.columns= [x.replace('.','_') for x in committees.columns]

committees= committees.drop(['subcommittees'], axis=1)

In [161]:
committees.head(3)

,chamber,committeetypecode,name,systemcode,url,parent_name,parent_systemcode,parent_url
0,House,Other,Bicentenary Committee,hcza00,https://api.congress.gov/v3/committee/house/hc...,NaN,NaN,NaN
1,House,Standing,Energy (Ad Hoc) Committee,hhah00,https://api.congress.gov/v3/committee/house/hh...,NaN,NaN,NaN
2,House,Select,U.S. Role in Iranian Arms Committee,hlbz00,https://api.congress.gov/v3/committee/house/hl...,NaN,NaN,NaN


# committee_members

In [162]:
committee_members.columns= [x.lower() for x in committee_members.columns]
committee_members.columns= [x.replace('.','_') for x in committee_members.columns]


committee_members

,committee_code,rank,title,bioguide
0,HSII,1.0,Chair,W000821
1,HSII,1.0,Ranking Member,G000551
2,HSII,2.0,NaN,L000564
3,HSII,2.0,NaN,N000179
4,HSII,3.0,NaN,W000804
...,...,...,...,...
3861,HSBA10,6.0,NaN,V000130
3862,HSBA10,7.0,NaN,N000193
3863,HSBA10,7.0,NaN,G000583
3864,HSBA10,8.0,NaN,D000594
